# Market Basket Analysis: Data Loading and Cleaning

## Setting Work Directory

In [3]:
import os
os.chdir('../')  # Change working directory

## Reading Input Files

In [9]:
import pandas as pd
aisles = pd.read_csv('instacart_2017_05_01/aisles.csv')
departments = pd.read_csv('instacart_2017_05_01/departments.csv')
products = pd.read_csv('instacart_2017_05_01/products.csv')
orders = pd.read_csv('instacart_2017_05_01/orders.csv')
orders_products = pd.read_csv('instacart_2017_05_01/order_products__prior.csv')

## Data Check
#### (shape, features, unique values and missing values)

In [10]:
print ("Rows     : " ,aisles.shape[0])
print ("Columns  : " ,aisles.shape[1])
print ("\nFeatures : \n" ,aisles.columns.tolist())
print ("\nMissing values :  ", aisles.isnull().sum().values.sum())
print ("\nUnique values :  \n",aisles.nunique())

Rows     :  134
Columns  :  2

Features : 
 ['aisle_id', 'aisle']

Missing values :   0

Unique values :  
 aisle_id    134
aisle       134
dtype: int64


In [11]:
print ("Rows     : " ,departments.shape[0])
print ("Columns  : " ,departments.shape[1])
print ("\nFeatures : \n" ,departments.columns.tolist())
print ("\nMissing values :  ", departments.isnull().sum().values.sum())
print ("\nUnique values :  \n",departments.nunique())

Rows     :  21
Columns  :  2

Features : 
 ['department_id', 'department']

Missing values :   0

Unique values :  
 department_id    21
department       21
dtype: int64


In [12]:
print ("Rows     : " ,products.shape[0])
print ("Columns  : " ,products.shape[1])
print ("\nFeatures : \n" ,products.columns.tolist())
print ("\nMissing values :  ",products.isnull().sum().values.sum())
print ("\nUnique values :  \n",products.nunique())

Rows     :  49688
Columns  :  4

Features : 
 ['product_id', 'product_name', 'aisle_id', 'department_id']

Missing values :   0

Unique values :  
 product_id       49688
product_name     49688
aisle_id           134
department_id       21
dtype: int64


In [13]:
print ("Rows     : " ,orders.shape[0])
print ("Columns  : " ,orders.shape[1])
print ("\nFeatures : \n" ,orders.columns.tolist())
print ("\nMissing values :  ", orders.isnull().sum().values.sum())
print ("\nUnique values :  \n",orders.nunique())

Rows     :  3421083
Columns  :  7

Features : 
 ['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']

Missing values :   206209

Unique values :  
 order_id                  3421083
user_id                    206209
eval_set                        3
order_number                  100
order_dow                       7
order_hour_of_day              24
days_since_prior_order         31
dtype: int64


In [14]:
print ("Rows     : " ,orders_products.shape[0])
print ("Columns  : " ,orders_products.shape[1])
print ("\nFeatures : \n" ,orders_products.columns.tolist())
print ("\nMissing values :  ", orders_products.isnull().sum().values.sum())
print ("\nUnique values :  \n",orders_products.nunique())

Rows     :  32434489
Columns  :  4

Features : 
 ['order_id', 'product_id', 'add_to_cart_order', 'reordered']

Missing values :   0

Unique values :  
 order_id             3214874
product_id             49677
add_to_cart_order        145
reordered                  2
dtype: int64
